<a href="https://colab.research.google.com/github/betoborda/pucp-pry-analisis-datos/blob/main/dashboard_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import functools
import io

import IPython
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

# ipywidgets
import ipywidgets as widgets

from ipywidgets import Button, Label, Image
from ipywidgets import Box, VBox, HBox
from ipywidgets import AppLayout, Layout

In [20]:
# Variables globales sobre el dataset

np.random.seed(42)

model = {}

num_columns = ['COSTO ALQUILER DIARIO', 'TOTAL ALQUILER', 'COSTO CONSERJE DIARIO', 
               'TOTAL CONSERJE' ,'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES',
               'LIQUIDACION EN DOLARES']

model["df_original"] = None
model["df_final"] = None
model["df"] = None
model["main_output"] = None
model["num_columns"] = num_columns
model["df_reglas"] = None

model['ciudad']=None
model['tipo']=None
model['categoria']=None
model['dia_in']=None
model['hora_in']=None
model['dia_out']=None
model['hora_out']=None
model['moneda']=None
model['alquiler_diario']=None
model['dias_conserje']=None
model['conserje_diario']=None
model['adicionales_soles']=None
model['agente']=None
model['estado_transaccion']=None

df_errores = widgets.Output()

In [21]:
###########
#. Errores (TODO)
###########

download_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

#Widgets para dataframe errores
# TODO renombar variable output
df_errores = widgets.Output()

#funcion graficar pie
def show_error_graph(pie_output):
  global model

  with pie_output:
    clear_output()
    dat = [model['df'].shape[0], model['df_final'].shape[0]]
    index=['Errores','Validos']
    plt.pie(dat, labels=index,autopct='%1.1f%%',)
    plt.title('CALIDAD DE LA DATA')
    plt.show()


#Funcionalidad N°8: Mostrar en el otro tab df los errores (esta dentro de la funcion reglas)
def show_errors(df):
    columns = ['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']
    with df_errores:
        clear_output()
        display(df[columns])


def download_file(d):
    print("download df.xlsx", df.shape)
    df.to_excel("df.xlsx")


# Subscribir eventos
download_btn.on_click(download_file)


calidad_items = [df_errores]
calidad_column = widgets.VBox(calidad_items)

error_tab = widgets.VBox(
    children=[
        widgets.Label("Errores destacados"),
        download_btn,
        calidad_column]
)

In [22]:
##############
# Carga Page
##############

def process_rules(checkboxes):
  global model,df1_final

  df = model['df_original'].copy()

  if checkboxes[0].value:
    df['valcostoalquiler_REGLA1'] = np.where(~np.isclose(df['COSTO ALQUILER DIARIO']*df['DIAS ALQUILER'],df['TOTAL ALQUILER']), 1, 0)
  else:
    df['valcostoalquiler_REGLA1'] = 0
    
  if checkboxes[1].value:
    df['valcostoconserje_REGLA2'] = np.where(~np.isclose(df['DIAS CONSERJE']*df['COSTO CONSERJE DIARIO'],df['TOTAL CONSERJE']), 1, 0)
  else:
    df['valcostoconserje_REGLA2'] = 0

  if checkboxes[2].value:
    df['valdiasalquiler_REGLA3'] = np.where(~np.isclose(df['DIAS ALQUILER'], (df['CHECK OUT'] - df['CHECK IN']).dt.ceil('d').dt.days), 1, 0)
  else:
    df['valdiasalquiler_REGLA3'] = 0

  if checkboxes[3].value:
    df['valcheckout_REGLA4'] = np.where((df['CHECK OUT'].isnull()) & (df['ESTADO TRANSACCION']!='ANULADO'), 1, 0)
  else:
    df['valcheckout_REGLA4'] = 0

  if checkboxes[4].value:
    df['valadicionalusd_REGLA5'] = np.where((df['MONEDA']=='USD') & (df['COSTOS ADICIONALES SOLES']!=0), 1, 0)
  else:
    df['valadicionalusd_REGLA5'] = 0

  if checkboxes[5].value:
    df['valsumacostosusd_REGLA6'] = np.where((df['MONEDA']=='USD') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valsumacostosusd_REGLA6'] = 0

  if checkboxes[6].value:
    df['valsumacostospen_REGLA7'] = np.where((df['MONEDA']=='PEN') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE']+df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valsumacostospen_REGLA7'] = 0

  if checkboxes[7].value:
    df['valpenusdpen_REGLA8'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valpenusdpen_REGLA8'] = 0

  if checkboxes[8].value:
    df['valpenusdusd_REGLA9'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER'] + df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valpenusdusd_REGLA9'] = 0
  
  df['errores'] = np.where((df['valcostoalquiler_REGLA1']==1) | 
                           (df['valcostoconserje_REGLA2']==1) | 
                           (df['valdiasalquiler_REGLA3']==1) | 
                           (df['valcheckout_REGLA4']==1) | 
                           (df['valadicionalusd_REGLA5']==1) | 
                           (df['valsumacostosusd_REGLA6']==1) | 
                           (df['valsumacostospen_REGLA7']==1) | 
                           (df['valpenusdpen_REGLA8']==1)| 
                           (df['valpenusdusd_REGLA9']==1), 1, 0)
  
   #### Añadir la columna Regla en el df(errores)
  conditions = [
    (df['valcostoalquiler_REGLA1'] ==1),
    (df['valcostoconserje_REGLA2']==1),
    (df['valdiasalquiler_REGLA3'] ==1),
    (df['valcheckout_REGLA4']==1),
    (df['valadicionalusd_REGLA5']==1),
    (df['valsumacostosusd_REGLA6']==1),
    (df['valsumacostospen_REGLA7']==1),
    (df['valpenusdpen_REGLA8']==1),
    (df['valpenusdusd_REGLA9']==1)]
  choices = ['Regla_1', 'Regla_2', 'Regla_3','Regla_4','Regla_5','Regla_6','Regla_7','Regla_8','Regla_9',]
  df['REGLA'] = np.select(conditions, choices)

  # TODO actualizar logica
  model['df_final'] = df[df['errores']==0].iloc[:,0:19] ### Df limpia
  model["df_reglas"]=df[df['errores']==1].iloc[:,0:30]

  df = df[df['errores']==1].iloc[:,0:19] #### Df con errores
  model['df'] = df

  df1_final=model['df_final'].copy()


def on_choose_file_change(change):
  pass


def on_upload_file_clicked(b, choose_file_btn, filename_txt):
  global model

  def leerArchivoCSV(botonCargar, nom_arch):
    return pd.read_excel(io.BytesIO(botonCargar.value[nom_arch]['content']))

  csv_nombre = next(iter(choose_file_btn.value))
  model["df_original"]=leerArchivoCSV(choose_file_btn, csv_nombre)
  filename_txt.value=csv_nombre


def on_process_file_clicked(b, rule_chk_list, pie_out):
  process_rules(rule_chk_list)
  #process_imputers()
  show_error_graph(pie_out)
  # TODO mostrar los errores del procesamiento en la otra pestaña
  #show_errors(df) 


def create_carga_buttons():
  choose_file_btn = widgets.FileUpload(
    accept='xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    icon = 'upload'
  )
  file_name_txt = widgets.Text(
    placeholder='Ningún archivo cargado',
    disabled=True
  )
  upload_file_btn = widgets.Button(
    description='subir archivo',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  process_file_btn = widgets.Button(
    description='Procesar',
    button_style='success')

  upload_file_btn.on_click(functools.partial(on_upload_file_clicked, 
                                             choose_file_btn=choose_file_btn,
                                             filename_txt=file_name_txt))
  
  return widgets.HBox([choose_file_btn, 
                      file_name_txt, 
                      upload_file_btn, 
                      process_file_btn])


# panel = pestaña
def create_carga_tab_panel():

  rule_names = ['R1: total alquiler = días alquiler * costo alquiler diario', 
                'R2: total conserje = días conserje * costo conserje diario', 
                'R3: días alquiler = check out - check in', 
                'R4: Si check out vacío => estado transacción = Anulado', 
                'R5: Si moneda USD => no hay costos adicionales soles', 
                'R6: Si moneda USD => Total alquiler + Total conserje == liquidación en dólares', 
                'R7: Si moneda PEN => Total alquiler + Total conserje + costos adicionales soles == liquidación soles', 
                'R8: Si moneda PEN/USD => costos adicionales soles == liquidación en soles', 
                'R9: Si moneda PEN/USD => Total alquiler + Total conserje == liquidación en dólares']

  def create_checkbox(description):
    #TODO crear un un checkbox, un label para la regla y agruparlos
    return widgets.Checkbox(value=True, description=description, indent=False)

  rule_checkboxes = [create_checkbox(description=rule_name) for rule_name in rule_names]

  consistencia_column = widgets.VBox(children=[widgets.Label("Consistencia"),
                                     *rule_checkboxes], 
                                     layout=Layout(width='auto'))

  limpieza_items = [widgets.Label("Limpieza"), 
                    widgets.Checkbox(description="Nulos", disabled=True) ]
  limpieza_column = widgets.VBox(limpieza_items)

  #Widgets para pie
  pie_output = widgets.Output()

  calidad_items = [widgets.Label("Calidad de los datos"), pie_output]
  calidad_column = widgets.VBox(calidad_items)

  carga_buttons_hbox = create_carga_buttons()
  process_file_btn = carga_buttons_hbox.children[3]
  process_file_btn.on_click(functools.partial(on_process_file_clicked,
                                              rule_chk_list=rule_checkboxes,
                                              pie_out=pie_output))

  return widgets.VBox(
    children=[
        carga_buttons_hbox,
        widgets.HBox(
            children=[consistencia_column, 
                      limpieza_column, 
                      calidad_column],
            layout=Layout(width='auto')
        )
    ]
  )

####################PESTAÑA ERRORES



def show_errors(df):
    #columns = ['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
     #          'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
     #          'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']
    with df_errores:
        clear_output()
        display(df)

def filter_error_rules(a):
    global df1
    with df_errores:
      clear_output()
      if dropdown_rule_btn.value=='Regla 1':
        df1=df[df['REGLA']=='Regla_1']
      elif dropdown_rule_btn.value=='Regla 2':
        df1=df[df['REGLA']=='Regla_2']
      elif dropdown_rule_btn.value=='Regla 3':
        df1=df[df['REGLA']=='Regla_3']
      elif dropdown_rule_btn.value=='Regla 4':
        df1=df[df['REGLA']=='Regla_4']
      elif dropdown_rule_btn.value=='Regla 5':
        df1=df[df['REGLA']=='Regla_5']
      elif dropdown_rule_btn.value=='Regla 6':
        df1=df[df['REGLA']=='Regla_6']
      elif dropdown_rule_btn.value=='Regla 7':
        df1=df[df['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 7':
        df1=df[df['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 8':
        df1=df[df['REGLA']=='Regla_8']
      elif dropdown_rule_btn.value=='Regla 9':
        df1=df[df['REGLA']=='Regla_9']
      else:
        df1=df
      return display(df1)

def create_error_tab_panel():

  dropdown_rule_btn=widgets.Dropdown(
    placeholder='Choose Someone',
    options=['Todos','Regla 1', 'Regla 2', 'Regla 3','Regla 4','Regla 5','Regla 6','Regla 7','Regla 8','Regla 9',],
    description='Filtro:',
    ensure_option=True,
    disabled=False)
  
  filter_rule_btn = widgets.Button(
    description='Filtrar',
    disabled=False,
    button_style='success',
    icon='check')
  
  download_error_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
    )


  def filter_error_rules(a):
    global df1
    with df_errores:
      clear_output()
      if dropdown_rule_btn.value=='Regla 1':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_1']
      elif dropdown_rule_btn.value=='Regla 2':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_2']
      elif dropdown_rule_btn.value=='Regla 3':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_3']
      elif dropdown_rule_btn.value=='Regla 4':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_4']
      elif dropdown_rule_btn.value=='Regla 5':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_5']
      elif dropdown_rule_btn.value=='Regla 6':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_6']
      elif dropdown_rule_btn.value=='Regla 7':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 7':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_7']
      elif dropdown_rule_btn.value=='Regla 8':
        df1= model['df_reglas'] [model['df_reglas'] ['REGLA']=='Regla_8']
      elif dropdown_rule_btn.value=='Regla 9':
        df1= model['df_reglas'] [ model['df_reglas'] ['REGLA']=='Regla_9']
      else:
        df1= model['df_reglas']
      return display(df1.iloc[:,0:19])
  
  def download_error_file(a):
    #print("download df.xlsx", df.shape)
    df1.iloc[:,0:19].to_excel("df.xlsx")

  #error_buttons_hbox = create_error_buttons()
  #dropdown_rule_btn = error_buttons_hbox.children[0]
  #filter_rule_btn = error_buttons_hbox.children[1]
  #process_file_btn.on_click(functools.partial(on_process_file_clicked,
  #                                            rule_chk_list=rule_checkboxes,
  #                                            pie_out=pie_output))

  filter_rule_btn.on_click(filter_error_rules)
  download_error_btn.on_click(download_error_file)

  calidad_items = [df_errores]
  calidad_column = widgets.VBox(calidad_items)
  

  return widgets.VBox(
    children=[widgets.Label("Errores destacados"),
        widgets.HBox([dropdown_rule_btn,filter_rule_btn,download_error_btn]),calidad_column
    ]
  )









def create_carga_page():
  # TODO añadir la pestaña error
  #children = [data_tab, error_tab]
  children = [create_carga_tab_panel(),create_error_tab_panel()]
  
  carga_tab = widgets.Tab()
  carga_tab.children = children
  carga_tab.set_title(0, 'Data')
  carga_tab.set_title(1, 'Errores')
  
  return VBox([Label(value="Carga de Datos"), 
               carga_tab])


In [23]:
######################
# Correlacion Page
######################

def on_correlacion_dropdown_x_changed(change, dropdown_y, corr_output):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return
  #model['conserje_diario']=change['new']
  num_x = model["num_columns"].index(change.new)
  x_idx=num_columns[num_x]
  num_y = model["num_columns"].index(dropdown_y.value)
  y_idx=num_columns[num_y]
  show_correlacion_graph(corr_output, x_idx, y_idx)


def on_correlacion_dropdown_y_changed(change, dropdown_x, corr_output):
  global model

  # Hack: porque el método se invoca muchas veces
  if change['name'] != 'value':
    return

  num_x = model["num_columns"].index(dropdown_x.value)
  x_idx =num_columns[num_x]
  num_y = model["num_columns"].index(change.new)
  y_idx = num_columns[num_y]
  show_correlacion_graph(corr_output, x_idx, y_idx)

def show_correlacion_graph(corr_output, x_idx, y_idx):

  global model
  df = model['df_final']
  with corr_output:
    clear_output()
    plt.scatter(df[x_idx], df[y_idx])
    plt.title('SCATTER')
    plt.xlabel(x_idx)
    plt.ylabel(y_idx);
    plt.show()



def create_correlacion_tab_panel():
  # Creación de widgets.Dropdown() para el scatterplot
  correlacion_x_dropdown=widgets.Dropdown(
    #placeholder='Choose Someone',
    options=num_columns,
    value=num_columns[0],
    description='Variable 1:'
    )
    #ensure_option=True,
    #disabled=False)
  correlacion_y_dropdown=widgets.Dropdown(
    #placeholder='Choose Someone',
    options=num_columns,
    value=num_columns[1],
    description='Variable 2:'
    )
    #ensure_option=True,
    #disabled=False)
  
  correlacion_output=widgets.Output()
  
  correlacion_x_dropdown.observe(functools.partial(on_correlacion_dropdown_x_changed,
                                              dropdown_y=correlacion_y_dropdown,
                                              corr_output=correlacion_output
                                              ))

  correlacion_y_dropdown.observe(functools.partial(on_correlacion_dropdown_y_changed,
                                              dropdown_x=correlacion_x_dropdown,
                                              corr_output=correlacion_output
                                              ))
  # default preview
  show_correlacion_graph(correlacion_output, num_columns[0],num_columns[1])
                     #model["num_columns"].index(num_columns[0]), 
                     #model["num_columns"].index(num_columns[1]))
  
  return widgets.VBox(
    children=[widgets.HBox([correlacion_x_dropdown, 
                      correlacion_y_dropdown]
                          ),
                      correlacion_output
              ]
            )

def create_correlacion_page():
  children1= [create_correlacion_tab_panel()]
  correlacion_tab = widgets.Tab()
  correlacion_tab.children = children1
  correlacion_tab.set_title(0, 'Correlación')
  
  return VBox([Label(value="Correlación de datos"), correlacion_tab])

In [24]:
################
# Outlier Page
################

def on_outlier_dropdown_x_changed(change, dropdown_y, outlier_output):
    global model

    # Hack: porque el método se invoca muchas veces
    if change['name'] != 'value':
      return

    x_idx = model["num_columns"].index(change.new)
    y_idx = model["num_columns"].index(dropdown_y.value)
    show_outlier_graph(outlier_output, x_idx, y_idx)


def on_outlier_dropdown_y_changed(change, dropdown_x, outlier_output):
    global model

    # Hack: porque el método se invoca muchas veces
    if change['name'] != 'value':
      return

    x_idx = model["num_columns"].index(dropdown_x.value)
    y_idx = model["num_columns"].index(change.new)
    show_outlier_graph(outlier_output, x_idx, y_idx)


def show_outlier_graph(outlier_output, x_idx, y_idx):
  global model
  
  df = model['df']
  ct = ColumnTransformer([('entre-0-1', StandardScaler(), num_columns)], 
                         remainder='drop')
  X = ct.fit_transform(df)

  clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

  y_pred = clf.fit_predict(X)
  X_scores = clf.negative_outlier_factor_

  with outlier_output:
    clear_output()
    plt.title("Local Outlier Factor (LOF)")
    plt.scatter(X[:, x_idx], X[:, y_idx], color='k', s=3., label='Data points')

    radius = (X_scores.max() - X_scores) / (X_scores.max() - X_scores.min())
    plt.scatter(X[:, x_idx], X[:, y_idx], s=1000 * radius, edgecolors='r', 
              facecolors='none', label='Outlier scores')
    plt.axis('tight')
    plt.xlim((-5, 5))
    plt.ylim((-5, 5))
    # plt.xlabel("prediction errors: %d" % (n_errors))
    plt.xlabel("prediction errors")
    legend = plt.legend(loc='upper left')
    legend.legendHandles[0]._sizes = [10]
    legend.legendHandles[1]._sizes = [20]
    plt.show()


def create_outlier_page():

  outlier_graph_x_dropdown = widgets.Dropdown(
    options=num_columns,
    value=num_columns[0],
    description='X field:',
  )

  outlier_graph_y_dropdown = widgets.Dropdown(
    options=num_columns,
    value=num_columns[1],
    description='Y field:',
  )
  outlier_scatter_output = widgets.Output()


  outlier_graph_x_dropdown.observe(functools.partial(on_outlier_dropdown_x_changed,
                                              dropdown_y=outlier_graph_y_dropdown,
                                              outlier_output=outlier_scatter_output
                                              ))

  outlier_graph_y_dropdown.observe(functools.partial(on_outlier_dropdown_y_changed,
                                              dropdown_x=outlier_graph_x_dropdown,
                                              outlier_output=outlier_scatter_output
                                              ))
  # default preview
  show_outlier_graph(outlier_scatter_output, 
                     model["num_columns"].index(num_columns[0]), 
                     model["num_columns"].index(num_columns[1]))

  outlier_page = widgets.VBox(
    children=[
        widgets.HBox(
            children=[Label(value="Parameters"),
                      outlier_graph_x_dropdown, 
                      outlier_graph_y_dropdown]
          ),
        outlier_scatter_output
      ]
    )

  children= [outlier_page]
  outlier_tab = widgets.Tab()
  outlier_tab.children = children
  outlier_tab.set_title(0, 'Outliers')

  return VBox([Label(value="Análisis de Outliers"), 
               outlier_tab])


In [25]:
################
# Register Page
################

def create_register_buttons():
  
  registrar_btn = widgets.Button(
    description='Registrar',
    disabled=False,
    button_style='success',
    icon='check')
  
  ciudades_list=['LIMA','CUZCO','TRUJILLO','LORETO','AREQUIPA','CAJAMARCA','TACNA']
  ciudad_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=ciudades_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CIUDAD:',
    ensure_option=True,
    disabled=False)
  def ciudad_change(change):
    global model
    model['ciudad']=change['new']
  ciudad_drop.observe(ciudad_change, names='value')

  tipo_list=['ALQUILER','USO INTERNO','CORTESIA']
  tipo_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=tipo_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='TIPO:',
    ensure_option=True,
    disabled=False)
  def tipo_change(change):
    global model
    model['tipo']=change['new']
  tipo_drop.observe(tipo_change, names='value')

  categoria_list=['DEPARTAMENTO','OFICINA','EVENTO','EVENTOS','LUJOS','STORAGE','CONSERJE']
  categoria_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=categoria_list,  #la variable options se carga al presionar boton procesar
    value=None,
    description='CATEGORIA:',
    ensure_option=True,
    disabled=False)
  def categoria_change(change):
    global model
    model['categoria']=change['new']
  categoria_drop.observe(categoria_change, names='value')

  file_dia_in_text = widgets.Text(
    #value='',
    placeholder='YYYY-MM-DD',
    description= 'DIA (IN):',
    disabled=False)
  def dia_in_change(change):
    global model
    model['dia_in']=change['new']
  file_dia_in_text.observe(dia_in_change, names='value')

  file_hora_in_text = widgets.Text(
    #value='',
    placeholder='HH:MM:SS',
    description= 'HORA (IN):',
    disabled=False)
  def hora_in_change(change):
    global model
    model['hora_in']=change['new']
  file_hora_in_text.observe(hora_in_change, names='value')

  file_dia_out_text = widgets.Text(
    #value='',
    placeholder='YYYY-MM-DD',
    description= 'DIA (OUT):',
    disabled=False)
  def dia_out_change(change):
    global model
    model['dia_out']=change['new']
  file_dia_out_text.observe(dia_out_change, names='value')

  file_hora_out_text = widgets.Text(
    #value='',
    placeholder='HH:MM:SS',
    description= 'HORA (OUT):',
    disabled=False)
  def hora_out_change(change):
    global model
    model['hora_out']=change['new']
  file_hora_out_text.observe(hora_out_change, names='value')

  moneda_list=['PEN','USD','PEN / USD']
  moneda_drop=widgets.Dropdown(
    placeholder='Choose Someone',
    options=moneda_list,  #la variable options se carga al presionar boton procesar
    value=None,
    #description='MONEDA:',
    ensure_option=True,
    disabled=False)
  def moneda_change(change):
    global model
    model['moneda']=change['new']
  moneda_drop.observe(moneda_change, names='value')

  file_alq_diario_text = widgets.Text(
    #value='',
    placeholder='COSTO ALQUILER DIARIO',
    #description= 'ALQ. DIARIO:',
    disabled=False)
  def alq_diario_change(change):
    global model
    model['alquiler_diario']=change['new']
  file_alq_diario_text.observe(alq_diario_change, names='value')

  file_dias_cons_text = widgets.Text(
    #value='',
    placeholder='NUM DÍAS CONSERJE',
    #description= 'DÍAS:',
    disabled=False)
  def dias_cons_change(change):
    global model
    model['dias_conserje']=change['new']
  file_dias_cons_text.observe(dias_cons_change, names='value')


  file_cons_diario_text = widgets.Text(
    #value='',
    placeholder='COSTO CONSERJE DIARIO',
    #description= 'CONS DIARIO:',
    disabled=False)
  def cons_diario_change(change):
    global model
    model['conserje_diario']=change['new']
  file_cons_diario_text.observe(cons_diario_change, names='value')

  file_cost_adicionales_text = widgets.Text(
    #value='',
    placeholder='COSTOS ADICIONALES SOLES',
    #description= 'ADICIONAL:',
    disabled=False)
  def adicionales_soles_change(change):
    global model
    model['adicionales_soles']=change['new']
  file_cost_adicionales_text.observe(adicionales_soles_change, names='value')

  file_agente_text = widgets.Text(
    #value='',
    placeholder='CODIGO AGENTE',
    #description= 'AGENTE:',
    disabled=False)
  def agente_change(change):
    global model
    model['agente']=change['new']
  file_agente_text.observe(agente_change, names='value')


  file_est_transaccion_text = widgets.Text(
    #value='',
    placeholder='ESTADO DE TRANSACCIÓN',
    #description= 'ESTADO:',
    disabled=False)
  def transaccion_change(change):
    global model
    model['estado_transaccion']=change['new']
  file_est_transaccion_text.observe(transaccion_change, names='value')
  
  ### \xa0 ES ESPACIO
  
  return widgets.VBox(children=[HBox([ciudad_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0MONEDA:'),moneda_drop]),
               HBox([tipo_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ALQUILER DIARIO:'),file_alq_diario_text]),
               HBox([categoria_drop,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0DÍAS CONSERJE:'),file_dias_cons_text]),
               HBox([file_dia_in_text,widgets.Label('COSTO CONSERJE DIARIO\xa0:'),file_cons_diario_text]),
               HBox([file_hora_in_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ADICIONALES SOLES:'),file_cost_adicionales_text]),
               HBox([file_dia_out_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0CÓDIGO AGENTE:'),file_agente_text]),
               HBox([file_hora_out_text,widgets.Label('\xa0\xa0\xa0\xa0\xa0\xa0ESTADO TRANSACCIÓN:'),file_est_transaccion_text]),
               HBox([registrar_btn])
                ])

def create_df_register_buttons():
  descargar_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success',
    icon='check')
  ver_datos_btn = widgets.Button(
    description='Ver',
    disabled=False,
    button_style='success',
    icon='check')
  return widgets.HBox([descargar_btn,ver_datos_btn])

def show_df_register(df_output):
  global df1_final
  #df1=registrar_fila
  with df_output:
    clear_output()
    display(df1_final[['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']])

def on_ver_datos_file_clicked(b,df_final):
  #df_final_output=widgets.Output()
  show_df_register(df_final)

def download_file(d):
    #print("download df.xlsx", df.shape)
    df1_final.to_excel("df.xlsx")

def create_df_register_tab_panel():
  df_register_buttons_Vbox=create_df_register_buttons()
  df_final_output=widgets.Output()
  ver_datos_file_btn=df_register_buttons_Vbox.children[1]
  ver_datos_file_btn.on_click(functools.partial(on_ver_datos_file_clicked,
                                              df_final=df_final_output))
  descargar_datos_file_btn=df_register_buttons_Vbox.children[0]
  descargar_datos_file_btn.on_click(download_file)
  return widgets.VBox([df_register_buttons_Vbox,df_final_output])


def create_register_tab_panel():
  register_buttons_hbox=create_register_buttons()
  registrar_file_btn = register_buttons_hbox.children[7].children[0]
  registrar_file_btn.on_click(registrar_fila)
  return register_buttons_hbox

def create_register_page():
  children2= [create_register_tab_panel(),create_df_register_tab_panel()]
  registrar_tab = widgets.Tab()
  registrar_tab.children = children2
  registrar_tab.set_title(0, 'Registrar')
  registrar_tab.set_title(1, 'Datos')

  return VBox([Label(value="Registrar datos"), registrar_tab])



###########################
# Funciones para registrar
###########################

def indicar_iata():
  global model
  
  ciudad=model['ciudad']
  if ciudad=='LIMA':
    a='LIM'
  elif ciudad=='CUZCO':
    a='CUZ'
  elif ciudad=='TRUJILLO':
    a='TRU'
  elif ciudad=='LORETO':
    a='LTO'
  elif ciudad=='AREQUIPA':
    a='AQP'
  elif ciudad=='CAJAMARCA':
    a='CJA'
  elif ciudad=='TACNA':
    a='TCQ'
  return a

def convertir_fecha(a,b):
  fecha=str(a)+" "+str(b)
  fecha=pd.Timestamp(fecha)
  return fecha

def calcular_dias(inicio,final):
  dias_alq=np.int64((final - inicio).ceil('d').days)
  return dias_alq

def tipo_moneda(alquiler,conserje,adicionales):
  global model
  if model['moneda']=='PEN':
    liquidacion_soles= alquiler + conserje + adicionales
    liquidacion_dolares= 0.0
  elif model['moneda']=='USD':
    liquidacion_soles= 0.0
    liquidacion_dolares= alquiler + conserje
  elif model['moneda']=='PEN / USD':
    liquidacion_soles= adicionales
    liquidacion_dolares= alquiler + conserje
  return liquidacion_soles,liquidacion_dolares

def registrar_fila(a):
  global model,df1_final
  
  ciudad=model['ciudad']
  iata=indicar_iata()
  tipo=model['tipo']
  categoria=model['categoria']
  file_dia_in_text=model['dia_in']
  file_hora_in_text=model['hora_in']
  file_dia_out_text=model['dia_out']
  file_hora_out_text=model['hora_out']
  check_in=convertir_fecha(file_dia_in_text,file_hora_in_text)
  check_out=convertir_fecha(file_dia_out_text,file_hora_out_text)
  moneda=model['moneda']
  dias_alquiler=calcular_dias(check_in,check_out)
  costo_alquiler=np.float64(model['alquiler_diario'])
  total_alquiler=np.round((calcular_dias(check_in,check_out))*(costo_alquiler),2)
  dias_conserje=np.int64(model['dias_conserje'])
  costo_conserje=np.float64(model['conserje_diario'])
  total_conserje=np.round((dias_conserje)*(costo_conserje),2)
  costos_adicionales_soles=np.float64(model['adicionales_soles'])
  costos_adicionales_dolares=''
  liquidacion_soles=tipo_moneda(total_alquiler,total_conserje,costos_adicionales_soles)[0]
  liquidacion_dolares=tipo_moneda(total_alquiler,total_conserje,costos_adicionales_soles)[1]
  agente=model['agente']
  estado_transaccion=model['estado_transaccion']
  data=np.array([[ciudad,iata,tipo,categoria,check_in,check_out,moneda,dias_alquiler,costo_alquiler,total_alquiler,dias_conserje,costo_conserje,total_conserje,costos_adicionales_soles,costos_adicionales_dolares,liquidacion_soles,liquidacion_dolares,agente,estado_transaccion]])
  pr=pd.DataFrame(data,columns=model['df_final'].columns)
  pr=pr.infer_objects() ### para que mantengan su tipo de dato
  pr['COSTOS ADICIONALES DOLARES']=pd.to_numeric(pr['COSTOS ADICIONALES DOLARES'])
  df1_final=pd.concat([df1_final,pr])
  df1_final.reset_index(drop=True, inplace=True)
  return df1_final



In [26]:
##############
# Main layout
##############

def create_logo_pucp_image():
  logo_url = "https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg"
  logo_image = IPython.display.Image(logo_url, 
                                     width=290)
  return widgets.Image(value=logo_image.data, 
                       format='jpg',
                       width=290,
                       height=145)


def create_main_title():
  title = "Detección de patrones de alquiler por ciudad"
  title_layout = Layout(width='60%', 
                        display='flex', 
                        flex_flow='column', 
                        align_items='center', 
                        justify_content='center')
  return widgets.Label(value=title, 
                       layout=title_layout)


def create_global_header():
  return HBox([create_main_title(), 
               create_logo_pucp_image()])


# Main output
def create_content_output():
  main_output = widgets.Output(layout={'border': '1px solid black'})
  model["main_output"] = main_output
  return main_output


#  Main Menu
def show_load_page(b):
  with model["main_output"]:
    clear_output()
    display(create_carga_page())

def show_correlation_page(b):
  with model["main_output"]:
    clear_output()
    display(create_correlacion_page())

def show_outliers_page(b):
  with model["main_output"]:
    clear_output()
    display(create_outlier_page())

def show_register_page(b):
  with model["main_output"]:
    clear_output()
    display(create_register_page())


def create_global_menu():
  items_layout = Layout(width='auto') # override the default width of the button to 'auto' to let the button grow
  main_menu_layout = Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='solid',
                            width='90%')
  
  menu_options = ['Carga', 'Correlación', 'Outliers','Registrar']
  main_menu_buttons = [Button(description=menu_option, layout=items_layout, button_style='success') for menu_option in menu_options]

  main_menu_buttons[0].on_click(show_load_page)
  main_menu_buttons[1].on_click(show_correlation_page)
  main_menu_buttons[2].on_click(show_outliers_page)
  main_menu_buttons[3].on_click(show_register_page)

  return Box(children=main_menu_buttons, 
             layout=main_menu_layout)

In [27]:
AppLayout(header=create_global_header(),
          left_sidebar=create_global_menu(),
          center=create_content_output(),
          pane_widths=[1, 5, 0])

AppLayout(children=(HBox(children=(Label(value='Detección de patrones de alquiler por ciudad', layout=Layout(a…